In [1]:
import os

In [2]:
%pwd

'c:\\Projekte\\Data_Science\\End-to-end-Machine-Learning-Project-with-MLflow\\Git_Project\\MLFlow-Machine-Learning\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Projekte\\Data_Science\\End-to-end-Machine-Learning-Project-with-MLflow\\Git_Project\\MLFlow-Machine-Learning'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    params: dict
    target_column: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            params = params,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from mlProject import logger
from mlProject.config.configuration import ConfigurationManager
from mlProject.models.model_creation import ModelCreator

import joblib

In [13]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        models = self.config.params
        for model_name in models.keys():
            model_config = models[model_name]
            model_creator = ModelCreator(model_config)
            model = model_creator.create_model(model_config)
            model.fit(train_x, train_y)
            joblib.dump(model, os.path.join(self.config.root_dir, model_name))

In [15]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-08-22 16:13:45,574: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-22 16:13:45,583: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-22 16:13:45,593: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-22 16:13:45,598: INFO: common: created directory at: artifacts]
[2024-08-22 16:13:45,600: INFO: common: created directory at: artifacts/model_trainer]
